In [ ]:
! pip install keras tensorflow-serving-api

In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.models import Model, Sequential
from keras.optimizers import RMSprop
from keras import backend as K
from keras.callbacks import TensorBoard

tf.__version__
# If not working use tensorflow 1.2 (as of 18.01.2018)

In [ ]:
# Create keras model

batch_size = 128
num_classes = 10
epochs = 20

sess = tf.Session()
K.set_session(sess)

# Keras layers can be called on TensorFlow tensors:
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=784))  # fully-connected layer with 128 units and ReLU activation
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
              metrics=['accuracy'])

In [ ]:
# Set up training and test data
from keras.datasets import mnist
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Train model
from time import time

# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                   callbacks=[tensorboard])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
import matplotlib.pyplot as plt
import numpy

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# one hidden layer with 128 nodes test acc: 9.796
# two hidden layers with 128(relu) 128(relu) nodes test acc: 0.981
# two hidden layers with 128(relu), 128(smax) nodes test acc: 0.973
# two hidden layers with 512(relu), 128(relu) nodes test acc: 0.983, val_acc: 0.9834

In [ ]:
# This is how the MNIST labes and data looks like
#print(y_test[800])
#print(x_test[800])

In [ ]:
# Save model as .pb (proto buffer)

K.set_learning_phase(0)  # all new operations will be in test mode from now on

from keras.models import model_from_config
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

###EXPORT Part
export_path = "gs://mkt-cloudml-jumpstart-christian/model_2_512_2"
#def to_savedmodel(model, export_path):
"""Convert the Keras HDF5 model into TensorFlow SavedModel."""

builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'input': model.inputs[0]},
                                    outputs={'income': model.outputs[0]})

with K.get_session() as sess:
   builder.add_meta_graph_and_variables(
        sess=sess,
        tags=[tag_constants.SERVING],
        signature_def_map={
            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature}
   )
builder.save()

In [ ]:
# get credentials for ML Engine calls
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from googleapiclient import errors

credentials = GoogleCredentials.get_application_default()

ml = discovery.build('ml','v1', credentials=credentials)
projectID = 'projects/{}'.format('mkt-cloudml-jumpstart')

In [ ]:
# Create ml_engine model
model_name = 'mnist_keras_test1'

# Use new versions instead
"""
requestDict = {'name': model_name,
               'description': 'Keras mnist classifier'}
try:
    response = ml.projects().models().create(parent=projectID,
                                         body=requestDict).execute()
    ml_model = response
    print(response)
except errors.HttpError, err:
    # Something went wrong, print out some information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())
"""    

In [ ]:
# Get latest version
# List ml_engine model version
requestDict = { }

model_path = projectID + "/models/" + model_name
try:
    response = ml.projects().models().versions().list(parent=model_path).execute()
    print(response)
    for vers in response["versions"]:
      print(vers["name"])
 
except errors.HttpError, err:
    # Something went wrong, print out some information.
    print('There was an error creating the model version. Check the details:')
    print(err._get_reason())   

In [ ]:
# Create ml_engine model version
version_name = "v3"
requestDict = { "name": version_name,
                "description": "521(relu),128(relu),10(smax) - RMSprop",
                "deploymentUri": export_path
              }

model_path = projectID + "/models/" + model_name
try:
    response = ml.projects().models().versions().create(parent=model_path ,
                                         body=requestDict).execute()
    print(response)
except errors.HttpError, err:
    # Something went wrong, print out some information.
    print('There was an error creating the model version. Check the details:')
    print(err._get_reason())   

In [ ]:
# Update ml_engine model version to be the default

version_path = projectID + "/models/" + model_name + "/versions/" + version_name

try:
    response = ml.projects().models().versions().setDefault(name=version_path, body={}).execute()
    print(response)
except errors.HttpError, err:
    # Something went wrong, print out some information.
    print('There was an error creating the model version. Check the details:')
    print(err._get_reason())   

In [ ]:
# Do online predition
ind = 6585
requestDict = { "instances": [ x_test[ind].tolist() ]
              }
#print(requestDict)
model_path = projectID + "/models/" + model_name
try:
    response = ml.projects().predict(name=model_path ,
                                         body=requestDict).execute()
    print(response)
except errors.HttpError, err:
    # Something went wrong, print out some information.
    print('There was an error predicting. Check the details:')
    print(err._get_reason())   

In [ ]:
# Get result

results_list = response["predictions"][0]["income"]
index = results_list.index(max(results_list))
print('Predicted digit: ' + str(index) + ' with a confidence of: ' + str(results_list[index]))

In [ ]:
# Show the actual input
import numpy as np
import matplotlib.pyplot as plt

pixels = np.array(x_test[ind], dtype='float32')

# Reshape the array into 28 x 28 array (2-dimensional array)
pixels = pixels.reshape((28, 28))

# Plot
label = ind
plt.title('Label is {label}'.format(label=label))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
# Run a number of online predictions
from random import randint

for x in range(0, 100):
    i = randint(0, 9999)
    requestDict = { "instances": [ x_test[i].tolist() ]
                  }
                   
    try:
        response = ml.projects().predict(name=model_path ,
                                         body=requestDict).execute()
        results_list = response["predictions"][0]["income"]
        index = results_list.index(max(results_list))
        print('For index: ' + str(i) + ' predicted digit is: ' + str(index) + ' with a confidence of: ' + str(results_list[index]))
    except errors.HttpError, err:
        # Something went wrong, print out some information.
        print('There was an error predicting. Check the details:')
        print(err._get_reason())  

In [ ]:
# Run a number of online predictions
from random import randint

a = randint(0, 9999)
b = randint(0, 9999)
print(y_test[a]) 
print(y_test[b])

requestDict = { "instances": [ 
                               { 
                                 "input": x_test[a].tolist()
                               },
                               { 
                                 "input": x_test[b].tolist()
                               }
                             ]
                  }
                   
try:
        response = ml.projects().predict(name=model_path ,
                                         body=requestDict).execute()
        #print(response)
        predictions_list = response["predictions"]
        for i, prediction in enumerate(predictions_list):
          results_list = prediction["income"]
          index = results_list.index(max(results_list))
          print(' For index ' + str(i) + ' the predicted digit is ' + str(index) + ' with a confidence of: ' + str(results_list[index]))
except errors.HttpError, err:
        # Something went wrong, print out some information.
        print('There was an error predicting. Check the details:')
        print(err._get_reason())  